In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
# Loading the movie dataset
movies_df = pd.read_csv('imdb_top_250.csv')


In [11]:
# Preprocessing and combining features
def combine_features(row):
    return f"{row['Plot']} {row['genre']} {row['directors']} {row['cast']}"

movies_df['combined_features'] = movies_df.apply(combine_features, axis=1)
movies_df['combined_features'] = movies_df['combined_features'].fillna('')


In [12]:
# Vectorization using TF-IDF features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['combined_features'])


In [13]:
# Creating recommender function based on user input
def recommend_movies(user_input, top_n=10):
    user_vec = tfidf.transform([user_input])
    cosine_sim = cosine_similarity(user_vec, tfidf_matrix).flatten()
    top_indices = cosine_sim.argsort()[-top_n:][::-1]

    recommendations = movies_df.iloc[top_indices][
        ['movie_name', 'year_of_release', 'IMDb_score', 'genre', 'Plot']
    ]
    recommendations['similarity_score'] = cosine_sim[top_indices]

    return recommendations.reset_index(drop=True)


In [16]:
# Example usage
user_preferences = input("What do you like watching??")
top_n = 5
recommendations = recommend_movies(user_preferences, top_n=top_n)


In [15]:
# Displaying the top N recommendations
print(f"Top {top_n} movie recommendations based on your preference ('{user_preferences}'):\n")
print(recommendations)

Top 5 movie recommendations based on your preference ('I like action movies set in space'):

                                          movie_name  year_of_release  \
0                                       Blade Runner             1982   
1                                     The Iron Giant             1999   
2                              It's a Wonderful Life             1946   
3                                    Bicycle Thieves             1948   
4  The Lord of the Rings: The Fellowship of the Ring             2001   

   IMDb_score                         genre  \
0         8.1         Action, Drama, Sci-Fi   
1         8.1  Animation, Action, Adventure   
2         8.6        Drama, Family, Fantasy   
3         8.3                         Drama   
4         8.9     Adventure, Drama, Fantasy   

                                                Plot  similarity_score  
0  A blade runner must pursue and terminate four ...          0.155860  
1  A young boy befriends a giant robot 

In [17]:
print(f"Top {top_n} movie recommendations based on your preference ('{user_preferences}'):\n")
print(recommendations)

Top 5 movie recommendations based on your preference ('I like horror movies with a touch of comedy'):

              movie_name  year_of_release  IMDb_score  \
0  It's a Wonderful Life             1946         8.6   
1           The Exorcist             1973         8.1   
2                  Alien             1979         8.5   
3              The Thing             1982         8.2   
4                 Psycho             1960         8.5   

                       genre  \
0     Drama, Family, Fantasy   
1                     Horror   
2             Horror, Sci-Fi   
3    Horror, Mystery, Sci-Fi   
4  Horror, Mystery, Thriller   

                                                Plot  similarity_score  
0  An angel is sent from Heaven to help a despera...          0.184503  
1  When a mysterious entity possesses a young gir...          0.115725  
2  After investigating a mysterious transmission ...          0.111787  
3  A research team in Antarctica is hunted by a s...          0.10984